<a href="https://colab.research.google.com/github/kate-markina2709/Lab/blob/main/KMZI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#from pylfsr import LFSR
import random
import scipy.special as sc
import numpy as np
import math
# алгоритмы тестов брались с: https://habr.com/ru/company/securitycode/blog/237695/

# отдельный LFSR генератор
def LFSR(a, S):
    S1 = []
    S1 = S.copy()
    tmp = []
    result = []
    while True:
        tmp.append(a[0] * S1[0] ^ a[1] * S1[1] ^ a[2] * S1[2]^ a[3] * S1[3]^ a[4] * S[4] ^ a[5] * S[5] ^ a[6] * S[6] ^ a[7] * S[7])
        result.append(S1.pop(0))
        S1.insert(3, tmp[-1])
        if S1 == S:
            break
    return result


# Комбинирование с прореживающим генератором
def Thinning_LFSR(a):
    LFSR_1 = []
    LFSR_2 = []
    S_1 = []
    S_2 = []
    res_seq = []

    for i in range(8):
        S_1.append(random.randint(0,1))
        S_2.append(random.randint(0, 1))

    LFSR_1 = LFSR(a, S_1)
    LFSR_2 = LFSR(a, S_2)

    for j in range(min(len(LFSR_1), len(LFSR_2))):
        if LFSR_2[j] == 1:
            res_seq.append(LFSR_1[j])

    return res_seq


def comparation(etalon_value, box, M, m):
    count = 0
    box_copy = box.copy()
    list_cont = list()
    box = box.tolist()
    for i in range(len(box)):
        if i + len(etalon_value) < len(box):
            if box[i:i + len(etalon_value)] == etalon_value:
                count += 1
    return count


# Тест перекрывающихся шаблонов
def overlapping_template_matching_test(eq, block_len=10, template_len=3, template=None):
    if template is None:
        template = [0, 0, 1]
    N = int(len(eq) / block_len)

    arr_1 = np.array(eq, dtype='int')
    arr_1 = arr_1.reshape(N, block_len)

    mat_ex = (block_len - template_len + 1) / (2 ** template_len)
    print('MO: ', mat_ex)

    sigmaSquare = block_len * ((1 / (2 ** template_len)) - ((2 * template_len - 1) / (2 ** (2 * template_len))))
    print('D: ', sigmaSquare)

    W_c = []
    for i in range(N):
        W_c.append(comparation(template, arr_1[i], block_len, template_len))

    XiSquare = 0

    for i in range(N):
        XiSquare += ((W_c[i] - mat_ex) ** 2) / sigmaSquare

    Pvalue = sc.gammaincc(N / 2, XiSquare / 2)
    # Последовательность считается случайной, если P-значение > 0.01.
    if Pvalue > 0.01:
        print('The sequence is random')
    else:
        print('The sequence is not random')


# Тест линейной сложности
def Linear_Complexity(S, M):
    Li = []
    Ti = []
    v = [0, 0, 0, 0, 0, 0, 0]
    pi = [0.010417, 0.03125, 0.125, 0.5, 0.25, 0.0625, 0.020833]
    tmp = []
    block = int(len(S)/M)

    arr_1 = np.array(S, dtype='int')
    arr_1 = arr_1.reshape(block, M)

    print('Wait!')
    for k in arr_1:
        c = [0 for i in range(M)]
        c[0] = 1
        b = [0 for i in range(M)]
        b[0] = 1
        L = 0
        m = -1
        N = 0
        while N < M:
            sum = 0
            for i in range(1, L + 1):
                sum = (sum + c[i] * k[N-i]) % 2
            d = (k[N] + sum) % 2
            if d == 1:
                t = [c[i] for i in range(M)]
                for j in range(M - N + m):
                    c[N - m + j] = (c[N - m + j] + b[j]) % 2
                if L <= N / 2:
                    L = N + 1 - L
                    m = N
                    b = [t[i] for i in range(M)]
            N += 1
        Li.append(L)

    M_ = M/2 + (((9+math.pow(-1, M-1)))/36) - (((M/3)+2/9)/(math.pow(2, M)))

    for l in Li:
        Ti.append(math.pow(-1, M)*(l-M_)+2/9)

    for t in Ti:
        if t <= -2.5:
            v[0] = v[0] + 1
        elif -2.5 < t <= -1.5:
            v[1] = v[1] + 1
        elif -1.5 < t <= -0.5:
            v[2] = v[2] + 1
        elif -0.5 < t <= 0.5:
            v[3] = v[3] + 1
        elif 0.5 < t <= 1.5:
            v[4] = v[4] + 1
        elif 1.5 < t <= 2.5:
            v[5] = v[5] + 1
        else:
            v[6] = v[6] + 1

    XiSquare = 0
    for k in range(7):
        XiSquare = XiSquare + math.pow(v[k] - M*pi[k], 2) / (M*pi[k])

    Pvalue = sc.gammainc(6 / 2, XiSquare / 2)

    # Последовательность считается случайной, если P-значение > 0.01.
    if Pvalue > 0.01:
        print('The sequence is random')
    else:
        print('The sequence is not random')

# Спектральный тест
def Spektr_test(S):
    n = len(S)
    S1 = []

    for i in S:
        if i == 0:
            S1.append(-1)
        else:
            S1.append(1)
    f = abs(np.fft.fft(S1))

    T = math.sqrt(math.log(1/0.05)*n)
    N0 = 0.95*n/2
    d = (int(N0)-N0)/math.sqrt(n*0.95*0.05/4)
    Pvalue = sc.erfc(abs(d)/math.sqrt(2))

    if Pvalue > 0.01:
        print('The sequence is random')
    else:
        print('The sequence is not random')


def get_seq_e(file):
    e_eq = []
    M = 1000000
    with open(file) as f:
        for line in f:
            for s in line:
                if s != '\n' and s != ' ':
                    e_eq.append(s)
    if len(e_eq) > M:
        e_eq = e_eq[0:M]
    return np.array(e_eq, dtype=int)


def get_seq_pi(file):
    pi_eq = []
    M = 1000000
    with open(file) as f:
        for line in f:
            for s in line:
                if s != '\n' and s != ' ':
                    pi_eq.append(s)
    if len(pi_eq) > M:
        pi_eq = pi_eq[0:M]
    return np.array(pi_eq, dtype=int)


def Peroid_LFSR_Thinning(n, m):
    return ((2**n)-1) * (2**(m-1))


if __name__ == '__main__':
    # 1 этап. Предварительная проверка работоспобности тестов
    seq_pi = get_seq_pi("data.pi")
    seq_e = get_seq_e("data.e")

    print('------------------------------------------------------')
    print('The Discrete Fourier Transform (Spectral) Test')
    Spektr_test(seq_pi)
    Spektr_test(seq_e)
    print('------------------------------------------------------')

    print('------------------------------------------------------')
    print('The Overlapping Template Matching Test ')
    overlapping_template_matching_test(seq_pi, 10, 9, [1,1,1,1,1,1,1,1,1])
    overlapping_template_matching_test(seq_e, 10, 9, [1,1,1,1,1,1,1,1,1])
    print('------------------------------------------------------')

    print('------------------------------------------------------')
    print('The Linear Complexity Test ')
    Linear_Complexity(seq_pi, 500)
    Linear_Complexity(seq_e, 500)
    Linear_Complexity([1,0,0,1,0,0,1,0,1,1,1,0,0,1,0,1], 8)
    print('------------------------------------------------------')

    # 2 этап. Генерация последовательности
    f_1 = [1, 0, 0, 0, 1, 1, 1, 1]
    f_2 = [1, 0, 0, 0, 0, 0, 1, 1]
    f_3 = [1, 0, 0, 0, 1, 0, 0, 1]
    f=f_2
    sequence = []
    res = []
    while len(res) < 1000000:
        sequence = Thinning_LFSR(f)
        if len(res)+len(sequence)<=1000000:
            for i in sequence:
                res.append(i)
        else:
            break
    if len(res)< 1000000:
        for i in range(1000000-len(res)):
            res.append(0)

    print('Sequence: ' + str(res))

    # 3 этап. Проверка случайности сгенерированной последовательности
    print('------------------------------------------------------')
    print('The Discrete Fourier Transform (Spectral) Test')
    Spektr_test(res)
    print('------------------------------------------------------')

    print('------------------------------------------------------')
    print('The Overlapping Template Matching Test ')
    overlapping_template_matching_test(res)
    print('------------------------------------------------------')

    print('------------------------------------------------------')
    print('The Linear Complexity Test ')
    Linear_Complexity(res, 500)
    print('------------------------------------------------------')
